### Re-produces computations related to HeLa 10 replicates (Table S1)

#### Load saved embeddings and preprocess

In [ ]:
# requires(c('FNN','dplyr','matrixStats', 'limma', 'msImpute'))

In [1]:
query_embedding <- read.csv('~/ion_mobility/peptideprotonet_embedding_space_90Kto20KsplitTrain_epoch300_featuresScaled_HeLa10Reps2hr_noEvidenceTrain.csv')

refrence_data <- read.csv('~/ion_mobility/peptideprotonet_embedding_space_90Kto20KsplitTrain_epoch300_featuresScaled_EvidenceTrain.csv')

refrence_data$PrecursorID <- paste(refrence_data$PrecursorID, refrence_data$Leading.razor.protein, sep = "_")

# this is HeLa 10 data, although variable name is called tcell
tcells_evidence <- read.delim("/stornext/General/data/academic/lab_davis/prot/benchmarking/PXD014777/HeLa_10_replicates_2hr/evidence.txt")

# keep only MSMS evidence
tcells_evidence <- tcells_evidence[tcells_evidence$Type %in% "TIMS-MULTI-MSMS",]
tcells_evidence$PrecursorID <- paste(paste(tcells_evidence$Modified.sequence, tcells_evidence$Charge, sep=""),
                                     tcells_evidence$Leading.razor.protein, sep ="_")
tcells_evidence$Species <- "HeLa_query"

query <- query_embedding # grep("dim|Raw.file", colnames(data))
reference <- refrence_data #  grep("dim", colnames(data))
query$Raw.file.id <- as.numeric(as.factor(query$Raw.file))
tcells_evidence$Raw.file.id <- as.numeric(as.factor(tcells_evidence$Raw.file))


lc_ms_anchors  <- c("Raw.file.id", "Charge","m.z", "Mass", "Intensity","Retention.time")
identified_peptides <- FNN::get.knnx(query[, lc_ms_anchors], tcells_evidence[, lc_ms_anchors], k=1)
identified_peptides <- query[identified_peptides$nn.index,]
identified_peptides$PrecursorID <- tcells_evidence$PrecursorID
identified_peptides$Species <- tcells_evidence$Species


unidentified_peptides <- dplyr::anti_join(query, identified_peptides,
                                            by = lc_ms_anchors)



reference_combined <- rbind(reference[,c(grep("dim", colnames(reference), value = TRUE), "Charge", "PrecursorID", "Species")],
                            identified_peptides[,c(grep("dim", colnames(reference), value = TRUE), "Charge" ,"PrecursorID", "Species")])
                            
unidentified_peptides_latent <- unidentified_peptides[, grep("dim", colnames(unidentified_peptides))]


reference_latent <- reference_combined[, grep("dim", colnames(reference_combined))]
reference_charge <- reference_combined$Charge
reference_ident_labels <- reference_combined$PrecursorID
reference_ident_species <- reference_combined$Species


all_unidentified_peptides <- unidentified_peptides

In [2]:
### this chuck implements the two-pass approach

reference_combined <- rbind(reference[,c(grep("dim", colnames(reference), value = TRUE), "Charge", "PrecursorID", "Species")],
                            identified_peptides[,c(grep("dim", colnames(reference), value = TRUE), "Charge" ,"PrecursorID", "Species")])
                            
# reference_combined <- reference_combined[grep("ox|Deamidation|Acetyl|Oxidation|ac", reference_combined$PrecursorID, invert=TRUE),]







unidentified_peptides_latent <- unidentified_peptides[, grep("dim", colnames(unidentified_peptides))]
#k = 10





reference_combined_sub <- reference_combined[reference_combined$Species %in% c("HeLa","Yeast"), ] # c("HeLa","Human","Yeast")

# reference_combined_sub <- reference_combined
reference_latent <- reference_combined_sub[, grep("dim", colnames(reference_combined_sub))]
#reference_latent <- t(apply(reference_latent, 1, FUN=function(x) x/sqrt(sum(x^2))))
reference_charge <- reference_combined_sub$Charge
reference_ident_labels <- reference_combined_sub$PrecursorID
reference_ident_species <- reference_combined_sub$Species




study_embedding <- reference_combined[reference_combined$Species %in% c("HeLa_query","Yeast"), ] # c("HeLa","Human","Yeast")
study_latent <- study_embedding[, grep("dim", colnames(study_embedding))]
#study_latent <- t(apply(study_latent, 1, FUN=function(x) x/sqrt(sum(x^2)) ))
study_charge <- study_embedding$Charge
study_ident_labels <- study_embedding$PrecursorID
study_ident_species <- study_embedding$Species






tcells_evidence_dedupped <- tcells_evidence[, c("Raw.file","PrecursorID")]
tcells_evidence_dedupped <- tcells_evidence_dedupped[!duplicated(tcells_evidence_dedupped),]
# tcells_evidence_dedupped <- tcells_evidence_dedupped[grep("ox|Deamidation|Acetyl|Oxidation|ac", tcells_evidence_dedupped$PrecursorID, invert=TRUE),]

Use `computeStats()` for label (peptide identity propagation) and computing false transfers, coverage, new identifications transferred etc 

In [3]:
computeStats <- function(thr, k){
  
  false_transfer_rate <- c()
  # coverage_per_run <- c()
  total_idents_per_run_after_transfer <- c()
  number_new_ident <- c()
  coverage_per_run_after_transfer <- c()
  coverage_per_run_before_transfer <- c()
  ident_list <- list()
  
  itr = 1
  
  for (run in unique(tcells_evidence_dedupped$Raw.file)){
    
    message(run)
    missing_idents <- setdiff(tcells_evidence_dedupped$PrecursorID,
                              tcells_evidence_dedupped$PrecursorID[tcells_evidence_dedupped$Raw.file == run])
  
    latent_study <- study_latent[study_embedding$PrecursorID %in% missing_idents,]
    s_charge <- study_charge[study_embedding$PrecursorID %in% missing_idents]
    s_ident_labels <- study_ident_labels[study_embedding$PrecursorID %in% missing_idents]
    s_ident_species <- study_ident_species[study_embedding$PrecursorID %in% missing_idents]
  
  
    run_query_peptides <- unidentified_peptides[unidentified_peptides$Raw.file %in% run,]
  
    run_query_peptides_latent <- run_query_peptides[, grep("dim", colnames(run_query_peptides))]
    #run_query_peptides_latent <- t(apply(run_query_peptides_latent, 1, FUN=function(x) x/sqrt(sum(x^2))))
    query_feature_charges <- run_query_peptides$Charge
    run_query_peptides_intensity <- run_query_peptides$Intensity
  
  
    knn_prototypes <- FNN::get.knnx(

        latent_study,
        run_query_peptides_latent,
      k = k) 
  
    probs <- exp(-0.5*((knn_prototypes$nn.dist^2)/matrixStats::rowSds(knn_prototypes$nn.dist^2)))
    # probs <- 1 - knn_prototypes$nn.dist^2
    ww <- matrix(s_charge[knn_prototypes$nn.index], nrow = nrow(probs), ncol = ncol(probs))
    charge <- matrix(query_feature_charges, nrow = nrow(ww), ncol = ncol(ww), byrow = FALSE)
    w <- ifelse(ww==charge, 1, 0)
  
  
  
  
  
    transferred_idents <- matrix(s_ident_labels[knn_prototypes$nn.index], 
                               nrow = nrow(probs), ncol = ncol(probs))
    transferred_species <- matrix(s_ident_species[knn_prototypes$nn.index], 
                                nrow = nrow(probs), ncol = ncol(probs))


    # probs <- w*probs
    # probs <- probs/rowSums(probs)
    # probs[!is.finite(probs)] <- 0

    transferred_idents[w==0] <- "other"
    transferred_species[w==0] <- "other"


    unmapped_queries <- (rowSums(probs < thr) == k)
    
    false_transfers_co <- which(rowSums(probs >= thr & transferred_species == c("Yeast") ) > 1)
    length(false_transfers_co)
  
  
    correct_transfers <-  (rowSums(probs >= thr & (transferred_species == "HeLa_query" | transferred_species == "HeLa") )> 2)
    correct_transfers_run_names <- run_query_peptides$Raw.file[correct_transfers]
    correct_transfer_idents <- transferred_idents[correct_transfers,]
    correct_transfer_idents <- apply(correct_transfer_idents,1, FUN=function(x) {
      z <- table(x)
      z <- z[!grepl("other", names(z))]
      names(z)[which.max(z)]}
      )
  
    correct_transfer_idents <- gsub("__","_",correct_transfer_idents)
    transfer_idents <- do.call(cbind, list(correct_transfers_run_names, correct_transfer_idents,
                                           run_query_peptides_intensity[correct_transfers]))
    transfer_idents <- data.frame(transfer_idents)
    colnames(transfer_idents) <- c("Raw.file","PrecursorID","Intensity")
    transfer_idents$PrecursorID <- gsub("__","_", transfer_idents$PrecursorID)
  
    
    
    ### computation of false transfers-------
    
    # false_transfer_rate[itr] <- length(false_transfers)
    
    query_feature_charges <- run_query_peptides$Charge[unmapped_queries]
    unmapped_queries_latent <- run_query_peptides_latent[unmapped_queries,]
    unmapped_queries_inetnsity <- run_query_peptides_intensity[unmapped_queries]
    
    if(nrow(unmapped_queries_latent) > 0 ) {
      
      
      ex_prototypes <- FNN::get.knnx(

        reference_latent,
        unmapped_queries_latent,
      k = k) 
  
    ex_probs <- exp(-0.5*((ex_prototypes$nn.dist^2)/matrixStats::rowSds(ex_prototypes$nn.dist^2)))
    # ex_probs <- 1 - ex_prototypes$nn.dist^2
    ww2 <- matrix(reference_charge[ex_prototypes$nn.index], nrow = nrow(ex_probs), ncol = ncol(ex_probs))
    charge_ex <- matrix(query_feature_charges, nrow = nrow(ww2), ncol = ncol(ww2), byrow = FALSE)
    w2 <- ifelse(ww2==charge_ex, 1, 0)
  
  
  
  
  
    transferred_idents <- matrix(reference_ident_labels[ex_prototypes$nn.index], 
                               nrow = nrow(ex_probs), ncol = ncol(ex_probs))
    transferred_species <- matrix(reference_ident_species[ex_prototypes$nn.index], 
                                nrow = nrow(ex_probs), ncol = ncol(ex_probs))


    #ex_probs <- w2*ex_probs
    #ex_probs <- ex_probs/rowSums(ex_probs)
    # ex_probs[!is.finite(ex_probs)] <- 0

    transferred_idents[w2==0] <- "other"
    transferred_species[w2==0] <- "other"
    
    
    false_transfers <- which(rowSums(ex_probs >= thr & transferred_species == c("Yeast")) > 1)
    length(false_transfers)
    
    
    
    new_transfers <- which(rowSums(ex_probs >= thr & (transferred_species == "HeLa") ) > 1)
    #table(new_transfers)
    
    if(length(new_transfers) > 0){
      
      new_transfers_run_names <- run_query_peptides$Raw.file[unmapped_queries][new_transfers]
      new_transfer_idents <- transferred_idents[new_transfers,, drop=FALSE]
      new_transfer_idents <- apply(new_transfer_idents,1, FUN=function(x) {
        z <- table(x)
        z <- z[!grepl("other", names(z))]
        names(z)[which.max(z)]
        }
      )
  
      new_transfer_idents <- gsub("__","_", new_transfer_idents)
      transfer_idents2 <- do.call(cbind, list(new_transfers_run_names, new_transfer_idents,
                                              unmapped_queries_inetnsity[new_transfers]))
      transfer_idents2 <- data.frame(transfer_idents2)
      colnames(transfer_idents2) <- c("Raw.file","PrecursorID","Intensity")
      transfer_idents2$PrecursorID <- gsub("__","_", transfer_idents2$PrecursorID)
  
      total_transferred_idents <- rbind(transfer_idents2, transfer_idents)
      number_new_ident[itr] <- length(setdiff(transfer_idents2$PrecursorID, tcells_evidence_dedupped$PrecursorID))
    } else {
      total_transferred_idents <- transfer_idents
      number_new_ident[itr] <- 0
    }
    
     
    false_transfers <- c(false_transfers, false_transfers_co)
    false_transfer_rate[itr] <- length(false_transfers)/length(union(total_transferred_idents$PrecursorID,
                                                                      tcells_evidence_dedupped$PrecursorID[tcells_evidence_dedupped$Raw.file %in% run]))
     
    } else{
      
      
      
      total_transferred_idents <- transfer_idents
      false_transfer_rate[itr] <- length(false_transfers_co)/length(union(total_transferred_idents$PrecursorID,
                                                                      tcells_evidence_dedupped$PrecursorID[tcells_evidence_dedupped$Raw.file %in% run]))
      number_new_ident[itr] <- 0
      
    }
    
     
    
    
    
    # need to store raw file, precursor and Intensity
    all_idents <- rbind(cbind(tcells_evidence[tcells_evidence$Raw.file %in% run, 
                                        c("Raw.file","PrecursorID","Intensity")],
                              type = "TIMS-MULTI-MSMS"),
                        cbind(total_transferred_idents, type = "PIP")
    )
    
    all_idents_in_run <- union(total_transferred_idents$PrecursorID,
                               tcells_evidence_dedupped$PrecursorID[tcells_evidence_dedupped$Raw.file %in% run])
    
    coverage_per_run_after_transfer[itr] <- length(intersect(all_idents_in_run, tcells_evidence_dedupped$PrecursorID))/ length(unique(tcells_evidence_dedupped$PrecursorID))
    
    
    coverage_per_run_before_transfer[itr] <- length(intersect(tcells_evidence_dedupped$PrecursorID[tcells_evidence_dedupped$Raw.file %in% run], tcells_evidence_dedupped$PrecursorID))/ length(unique(tcells_evidence_dedupped$PrecursorID))
    
    

    total_idents_per_run_after_transfer[itr] <- length(all_idents_in_run)
    ident_list[[itr]] <- data.frame(all_idents)

   
    
    itr = itr + 1
     
    
  
  }
  
  ALL_IDENTS <- do.call(rbind, ident_list)
  return(list("threshold" = rep(thr, length(unique(tcells_evidence_dedupped$Raw.file))), 
              "false_transfer_rate" = false_transfer_rate, 
                         #"median_coverage" = med_cov_per_run, 
              "coverage_before_transfer" = coverage_per_run_before_transfer,
              "coverage_after_transfer" = coverage_per_run_after_transfer,
              "number_new_idents" = number_new_ident,
              "total_idents_per_run_after_PIP" = total_idents_per_run_after_transfer,
              "identifications" = ALL_IDENTS))
}

Quantify PIPP for $k=10,5$ and various cutoffs:

In [4]:
############# Tabularise results for different cut-offs

results_k10 <- list()
itr = 1
for (thr in c(0, 0.01, 0.05, 0.2,0.5, 0.8)){
  res <- computeStats(thr, k = 10)
  results_k10[[itr]] <- list("threshold" = thr, "false_transfer_rate" = median(res$false_transfer_rate), 
                         "mean_coverage" = median(res$coverage_before_transfer), 
                         "mean_coverage_after_transfer" = mean(res$coverage_after_transfer),
                         "max_coverage_after_transfer" = max(res$coverage_after_transfer),
                         "N_new_idents" = round(median(res$number_new_idents)),
                         "median_idents_per_run_after_PIP" = round(median(res$total_idents_per_run_after_PIP)),
                         "identifications" = res$identifications)
  
  itr = itr + 1
  
}

coverage_results <- lapply(results_k10, FUN=function(x)  x[!grepl("identifications", names(x))])

(dt_k10 <- data.frame(do.call(rbind, coverage_results)))
dt_k10 <- data.frame(round(data.matrix(dt_k10),3))





results_5k <- list()
itr = 1
for (thr in c(0, 0.01, 0.05, 0.2,0.5, 0.8)){
  res <- computeStats(thr, k = 5)
  results_5k[[itr]] <- list("threshold" = thr, "false_transfer_rate" = median(res$false_transfer_rate), 
                         "mean_coverage" = median(res$coverage_before_transfer), 
                         "mean_coverage_after_transfer" = mean(res$coverage_after_transfer),
                         "max_coverage_after_transfer" = max(res$coverage_after_transfer),
                         "N_new_idents" = round(median(res$number_new_idents)),
                         "median_idents_per_run_after_PIP" = round(median(res$total_idents_per_run_after_PIP)),
                         "identifications" = res$identifications)
  
  itr = itr + 1
  
}

coverage_results <- lapply(results_5k, FUN=function(x)  x[!grepl("identifications", names(x))])

(dt_k5 <- data.frame(do.call(rbind, coverage_results)))
dt_k5 <- data.frame(round(data.matrix(dt_k5),3))




20190122_HeLa_QC_Slot1-47_1_3219

20190122_HeLa_QC_Slot1-47_1_3220

20190122_HeLa_QC_Slot1-47_1_3221

20190122_HeLa_QC_Slot1-47_1_3222

20190122_HeLa_QC_Slot1-47_1_3223

20190122_HeLa_QC_Slot1-47_1_3224

20190122_HeLa_QC_Slot1-47_1_3225

20190122_HeLa_QC_Slot1-47_1_3226

20190122_HeLa_QC_Slot1-47_1_3227

20190122_HeLa_QC_Slot1-47_1_3228

20190122_HeLa_QC_Slot1-47_1_3219

20190122_HeLa_QC_Slot1-47_1_3220

20190122_HeLa_QC_Slot1-47_1_3221

20190122_HeLa_QC_Slot1-47_1_3222

20190122_HeLa_QC_Slot1-47_1_3223

20190122_HeLa_QC_Slot1-47_1_3224

20190122_HeLa_QC_Slot1-47_1_3225

20190122_HeLa_QC_Slot1-47_1_3226

20190122_HeLa_QC_Slot1-47_1_3227

20190122_HeLa_QC_Slot1-47_1_3228

20190122_HeLa_QC_Slot1-47_1_3219

20190122_HeLa_QC_Slot1-47_1_3220

20190122_HeLa_QC_Slot1-47_1_3221

20190122_HeLa_QC_Slot1-47_1_3222

20190122_HeLa_QC_Slot1-47_1_3223

20190122_HeLa_QC_Slot1-47_1_3224

20190122_HeLa_QC_Slot1-47_1_3225

20190122_HeLa_QC_Slot1-47_1_3226

20190122_HeLa_QC_Slot1-47_1_3227

20190122_HeLa_

threshold,false_transfer_rate,mean_coverage,mean_coverage_after_transfer,max_coverage_after_transfer,N_new_idents,median_idents_per_run_after_PIP
<list>,<list>,<list>,<list>,<list>,<list>,<list>
0,0,0.6749633,0.8557953,0.8601684,0,54571
0.01,0.009013491,0.6749633,0.8558632,0.8602157,872,55452
0.05,0.01329514,0.6749633,0.8544497,0.8588891,1416,55921
0.2,0.01081507,0.6749633,0.8427913,0.8480551,1640,55408
0.5,0.003210259,0.6749633,0.7836337,0.7903315,723,50778
0.8,0.0001349155,0.6749633,0.700946,0.7106082,44,44854


20190122_HeLa_QC_Slot1-47_1_3219

20190122_HeLa_QC_Slot1-47_1_3220

20190122_HeLa_QC_Slot1-47_1_3221

20190122_HeLa_QC_Slot1-47_1_3222

20190122_HeLa_QC_Slot1-47_1_3223

20190122_HeLa_QC_Slot1-47_1_3224

20190122_HeLa_QC_Slot1-47_1_3225

20190122_HeLa_QC_Slot1-47_1_3226

20190122_HeLa_QC_Slot1-47_1_3227

20190122_HeLa_QC_Slot1-47_1_3228

20190122_HeLa_QC_Slot1-47_1_3219

20190122_HeLa_QC_Slot1-47_1_3220

20190122_HeLa_QC_Slot1-47_1_3221

20190122_HeLa_QC_Slot1-47_1_3222

20190122_HeLa_QC_Slot1-47_1_3223

20190122_HeLa_QC_Slot1-47_1_3224

20190122_HeLa_QC_Slot1-47_1_3225

20190122_HeLa_QC_Slot1-47_1_3226

20190122_HeLa_QC_Slot1-47_1_3227

20190122_HeLa_QC_Slot1-47_1_3228

20190122_HeLa_QC_Slot1-47_1_3219

20190122_HeLa_QC_Slot1-47_1_3220

20190122_HeLa_QC_Slot1-47_1_3221

20190122_HeLa_QC_Slot1-47_1_3222

20190122_HeLa_QC_Slot1-47_1_3223

20190122_HeLa_QC_Slot1-47_1_3224

20190122_HeLa_QC_Slot1-47_1_3225

20190122_HeLa_QC_Slot1-47_1_3226

20190122_HeLa_QC_Slot1-47_1_3227

20190122_HeLa_

threshold,false_transfer_rate,mean_coverage,mean_coverage_after_transfer,max_coverage_after_transfer,N_new_idents,median_idents_per_run_after_PIP
<list>,<list>,<list>,<list>,<list>,<list>,<list>
0,0,0.6749633,0.9034018,0.9064736,0,57634
0.01,0.01584701,0.6749633,0.9019899,0.9054786,2410,59947
0.05,0.01589596,0.6749633,0.8969472,0.9008512,2610,59876
0.2,0.008691126,0.6749633,0.8703312,0.8758509,1844,57438
0.5,0.002113092,0.6749633,0.7372495,0.7472481,567,47740
0.8,6.949685e-05,0.6749633,0.6787899,0.6878662,26,43368


Compute coefficient of variation for MQ+/- MBR

In [5]:
# Coefficient of variation for MQ+/- MBR
devtools::load_all("/stornext/General/data/academic/lab_davis/prot/benchmarking/msImpute/")

data <- read.delim("/stornext/General/data/academic/lab_davis/prot/benchmarking/PXD014777/HeLa_10_replicates_2hr/evidence.txt", 
                   stringsAsFactors = FALSE)


table(data$Type)

# data <- data[grep("CON_|REV_", data$Leading.razor.protein, invert=TRUE),]
# data <- data[data$Charge > 1,]
data$PeptideID <- paste0(data$Modified.sequence, data$Charge)
data$matrix.row.id <- paste(data$PeptideID, data$Leading.Razor.Protein, sep ="_")


genes <- data[,c("PeptideID","matrix.row.id", "Leading.razor.protein")]
genes <- genes[!duplicated(genes),]


y_noMBR <- evidenceToMatrix(data[data$Type %in% "TIMS-MULTI-MSMS",])
y_MBR <- evidenceToMatrix(data)

mean(complete.cases(y_noMBR))
mean(complete.cases(y_MBR))

dim(y_noMBR)  
dim(y_MBR)  


mean(colSums(!is.na(y_noMBR))/nrow(y_noMBR))
mean(colSums(!is.na(y_MBR))/nrow(y_MBR))

median(matrixStats::rowSds(y_noMBR, na.rm = TRUE)/rowMeans(y_noMBR, na.rm = TRUE), na.rm = TRUE)
median(matrixStats::rowSds(y_MBR, na.rm = TRUE)/rowMeans(y_MBR, na.rm = TRUE), na.rm = TRUE)


median(matrixStats::rowSds(y_noMBR[complete.cases(y_noMBR),])/rowMeans(y_noMBR[complete.cases(y_noMBR),]))
median(matrixStats::rowSds(y_MBR[complete.cases(y_MBR),], na.rm = TRUE)/rowMeans(y_MBR[complete.cases(y_MBR),]))



y_MBR <- y_MBR[!grepl("_\\(.*\\)", rownames(y_MBR)),]

y_MBR <- limma::normalizeBetweenArrays(log2(y_MBR), method = "quantile")

genes <- genes[match(rownames(y_MBR), genes$PeptideID),]




sumTopN <- function(x, n=10 , na.rm = TRUE){
  sum(sort(x, decreasing = TRUE)[1:n], na.rm=na.rm)
}

pIds <- genes$Leading.razor.protein



yprot_mbr <- aggregate(.~ ProteinID, FUN=sumTopN, data = data.frame(y_MBR, ProteinID = pIds), 
                       na.action = na.pass
                       #na.rm=TRUE, trim = 0.2
                       )
rownames(yprot_mbr) <- yprot_mbr$ProteinID
yprot_mbr$ProteinID <- NULL
yprot_mbr[yprot_mbr==0] <- NA



ℹ Loading msImpute




TIMS-MULTI-MATCH  TIMS-MULTI-MSMS 
          186399           448067 

[1] 0.3572861

[1] 0.3573267

[1] 63319    10

...

[1] 0.6699383

[1] 0.08857373

[1] 0.08858557

[1] 0.0694166

[1] 0.06944282

Compute coefficient of variation for PIP at each threshold

In [6]:
## Coefficient of variation for PIP at each threshold
cv_results_k10 <- list()
for (j in seq_along(results_k10)){
  z <- results_k10[[j]]$identifications
  z$Intensity <- as.numeric(z$Intensity)
  m <- evidenceToMatrix(z, peptide_id = "PrecursorID")
  print(table(!grepl("^_", rownames(m))))
  
  m <- m[grepl("^_", rownames(m)), ]
  m <- m[!grepl("_\\(.*\\)", rownames(m)),]
  # print(tail(m))
  m <- limma::normalizeBetweenArrays(log2(m), method = "quantile")
  
  # cv <- median(matrixStats::rowSds(m, na.rm = TRUE)/rowMeans(m, na.rm = TRUE), na.rm = TRUE)
  cv <- median(matrixStats::rowSds(m[complete.cases(m),], na.rm = TRUE)/rowMeans(m[complete.cases(m),]))
  
  # quantified proteins : number of proteins in at least two runs.
  pIds <- gsub("(.*)_([1-5])_(.*)","\\3", rownames(m))
  mprot <- aggregate(.~ ProteinID, FUN=sumTopN, data = data.frame(m, ProteinID = pIds), 
                       na.action = na.pass
                       #na.rm=TRUE, trim = 0.2
                       )
  rownames(mprot) <- mprot$ProteinID
  mprot$ProteinID <- NULL
  mprot[mprot==0] <- NA
  
  
  
  cv_results_k10[[j]] <- list("threshold" = results_k10[[j]]$thr[1],
                          "CV_peptide" = cv,
                          "total_idents" = nrow(m),
                          "complete_cases" = mean(complete.cases(m)),
                          "n_quant_proteins" = sum(rowSums(!is.na(mprot)) > 2))
}


cv_results_k10[[length(cv_results_k10) +1]] <- list("threshold" = "MQ+MBR", 
                                  "CV_peptide" = median(matrixStats::rowSds(y_MBR[complete.cases(y_MBR),], na.rm = TRUE)/rowMeans(y_MBR[complete.cases(y_MBR),])),
                                  "total_idents" = nrow(y_MBR),
                                  "complete_cases" = mean(complete.cases(y_MBR)),
                                  "n_quant_proteins" = sum(rowSums(!is.na(yprot_mbr)) > 2))

(cv_results_k10 <- do.call(rbind, cv_results_k10))
cv_results_k10 <- data.frame(cv_results_k10)
cv_results_k10 <- data.frame(round(data.matrix(cv_results_k10), 3))
cv_results_k10[is.na(cv_results_k10)] <- "MQ+MBR"


cv_results_k5 <- list()
for (j in seq_along(results_5k)){
  z <- results_5k[[j]]$identifications
  z$Intensity <- as.numeric(z$Intensity)
  m <- evidenceToMatrix(z, peptide_id = "PrecursorID")
  print(table(!grepl("^_", rownames(m))))
  
  m <- m[grepl("^_", rownames(m)), ]
  m <- m[!grepl("_\\(.*\\)", rownames(m)),]
  # print(tail(m))
  
  m <- limma::normalizeBetweenArrays(log2(m), method = "quantile")
  
  
  # cv <- median(matrixStats::rowSds(m, na.rm = TRUE)/rowMeans(m, na.rm = TRUE), na.rm = TRUE)
  cv <- median(matrixStats::rowSds(m[complete.cases(m),], na.rm = TRUE)/rowMeans(m[complete.cases(m),]))
   # quantified proteins : number of proteins in at least two runs.
  pIds <- gsub("(.*)_([1-5])_(.*)","\\3", rownames(m))
  mprot <- aggregate(.~ ProteinID, FUN=sumTopN, data = data.frame(m, ProteinID = pIds), 
                       na.action = na.pass
                       #na.rm=TRUE, trim = 0.2
                       )
  rownames(mprot) <- mprot$ProteinID
  mprot$ProteinID <- NULL
  mprot[mprot==0] <- NA
  
  
  
  
  cv_results_k5[[j]] <- list("threshold" = results_5k[[j]]$thr[1],
                          "CV_peptide" = cv,
                          "total_idents" = nrow(m),
                          "complete_cases" = mean(complete.cases(m)),
                          "n_quant_proteins" = sum(rowSums(!is.na(mprot)) > 2))
}


cv_results_k5[[length(cv_results_k5) +1]] <- list("threshold" = "MQ+MBR", 
                                  "CV_peptide" = median(matrixStats::rowSds(y_MBR[complete.cases(y_MBR),], na.rm = TRUE)/rowMeans(y_MBR[complete.cases(y_MBR),])),
                                  "total_idents" = nrow(y_MBR),
                                  "complete_cases" = mean(complete.cases(y_MBR)),
                                  "n_quant_proteins" = sum(rowSums(!is.na(yprot_mbr)) > 2))

(cv_results_k5 <- do.call(rbind, cv_results_k5))
cv_results_k5 <- data.frame(cv_results_k5)
cv_results_k5 <- data.frame(round(data.matrix(cv_results_k5), 3))
cv_results_k5[is.na(cv_results_k5)] <- "MQ+MBR"


# save(dt_k10, dt_k5, cv_results_k10, cv_results_k5, file = "HeLa_10rep_2hr_peptideprotonet_evals_normalisedIntensity_nquantprotein.RData")


FALSE 
64117 

FALSE 
68611 

FALSE 
70410 

FALSE 
70313 

FALSE 
67072 

FALSE 
63879 


threshold CV_peptide  total_idents complete_cases n_quant_proteins
[1,] 0         0.0236901   63101        0.7623334      5980            
[2,] 0.01      0.0235098   67526        0.7117851      6514            
[3,] 0.05      0.02229305  69286        0.6904281      6876            
[4,] 0.2       0.01716095  69203        0.6690461      6961            
[5,] 0.5       0.009268429 66006        0.5598279      6276            
[6,] 0.8       0.006262167 62859        0.4056698      5853            
[7,] MQ+MBR    0.005839133 62307        0.355241       5782

Warning message in data.matrix(cv_results_k10):
“NAs introduced by coercion”



FALSE 
64190 

FALSE 
73996 

FALSE 
73839 

FALSE 
70926 

FALSE 
66400 

FALSE 
63604 


threshold CV_peptide  total_idents complete_cases n_quant_proteins
[1,] 0         0.02551834  63174        0.8139424      6041            
[2,] 0.01      0.02387729  72841        0.7025302      7457            
[3,] 0.05      0.02112347  72685        0.6937883      7540            
[4,] 0.2       0.01403146  69819        0.6584454      7062            
[5,] 0.5       0.006676491 65347        0.4063691      6168            
[6,] 0.8       0.005871647 62585        0.3597348      5812            
[7,] MQ+MBR    0.005839133 62307        0.355241       5782

Warning message in data.matrix(cv_results_k5):
“NAs introduced by coercion”


In [12]:
(dt <- data.frame(rbind(cv_results_k5, cv_results_k10)))

threshold,CV_peptide,total_idents,complete_cases,n_quant_proteins
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.026,63174,0.814,6041
0.01,0.024,72841,0.703,7457
0.05,0.021,72685,0.694,7540
0.2,0.014,69819,0.658,7062
0.5,0.007,65347,0.406,6168
0.8,0.006,62585,0.360,5812
MQ+MBR,0.006,62307,0.355,5782
0,0.024,63101,0.762,5980
0.01,0.024,67526,0.712,6514


#### Session Information

In [15]:
sessionInfo()

R version 4.0.5 (2021-03-31)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS:   /stornext/System/data/apps/R/R-4.0.5/lib64/R/lib/libRblas.so
LAPACK: /stornext/System/data/apps/R/R-4.0.5/lib64/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] kableExtra_1.3.4 msImpute_1.7.1  

loaded via a namespace (and not attached):
  [1] bitops_1.0-6                matrixStats_0.57.0         
  [3] fs_1.5.0                    usethis_2.0.1              
  [5] devtools_2.4.2              w